<a href="https://colab.research.google.com/github/Anag0es/chatbot_langchain/blob/main/chatbot_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot com Langchain usando Panel e OpenAI

# ============================================



**Objetivo**: Neste notebook, construiremos um chatbot interativo utilizando as bibliotecas `Langchain` e `Panel`, integrando com a API da OpenAI.

A ideia é entender como criar um fluxo de conversa usando memória e modelos de linguagem, além de exibir a interface gráfica com `Panel`.

## Instalação das dependências

In [ ]:
!pip install -U langchain-community
!pip install openai
!pip install jupyter_bokeh

## Importação da biblioteca

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

import panel as pn
import os


## Configurando a API da OpenAI e dashboard

In [ ]:
llm = ChatOpenAI()
memory = ConversationBufferMemory()
os.environ["OPENAI_API_KEY"] = "sua_chave_api_aqui"
pn.extension()

## Criando cadeia de conversação

A `ConversationChain` é onde conectamos o modelo de linguagem e a memória.

In [ ]:
conversation = ConversationChain(
llm=llm,
memory=memory
)

## Configurando os Elementos da Interface

In [ ]:
# configurando o spinner para usuário aguardar resposta
loading_spinner = pn.indicators.LoadingSpinner(width=25, height=25, value=False)
loading_spinner.visible = False

### Definindo a Classe Chatbot

Agora, criamos uma classe `Chatbot` para encapsular a lógica de resposta e exibição do histórico de mensagens.

In [ ]:
class Chatbot:
def __init__(self):
self.conversation = conversation
self.history = []

def respond(self, message):
  if not message:
    return ""

  loading_spinner.visible = True
  loading_spinner.value = True
  response = self.conversation.run(message)
  self.history.append(("Usuário", message))
  self.history.append(("Bot", response))
  return self.display_chat()

def display_chat(self):
  chat_display = ""
  for speaker, msg in self.history:
    chat_display += f"**{speaker}:** {msg}\n\n"
  return chat_display

In [ ]:
chatbot = Chatbot()

In [ ]:
# Usamos widgets do Panel para criar os componentes de entrada e saída do chatbot.

input_box = pn.widgets.TextInput(placeholder="Digite sua mensagem aqui...")
send_button = pn.widgets.Button(name="Enviar", button_type="primary")
output = pn.pane.Markdown("")

In [ ]:
# Organizamos a interface como um dashboard vertical com título, saída e elementos de entrada.

dashboard = pn.Column(
pn.pane.Markdown("# Chatbot com Langchain"),
output,
pn.Row(input_box, send_button)
)

## Definindo o Comportamento do Botão

In [ ]:
# A função `send_message` é chamada quando o botão "Enviar" é clicado. Ela processa a entrada do usuário e exibe a resposta.

def send_message(event):
if input_box.value:
response = chatbot.respond(input_box.value)
output.object = response
input_box.value = ""

send_button.on_click(send_message)

In [ ]:
# Finalmente, criamos uma instância do chatbot e tornamos o dashboard visível.

dashboard.servable()
